<a href="https://colab.research.google.com/github/theAkhileshRai/Projects/blob/master/NN_BinaryClassifier_mean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder    


from sklearn.metrics import confusion_matrix, classification_report

from keras.utils import to_categorical

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler

In [0]:
df1 = pd.read_csv('/content/Combined_df.csv',index_col=False)

In [30]:
df1.head()

,Unnamed: 0,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,y_true,leak_level,station
0,0,05/04/19 2:30,1,0.666534,2.080444,3.220836,10.105964,10.063987,0.666534,2.053527,3.307406,10.142075,10.063987,1,2,Cross Barr
1,1,05/04/19 3:30,1,0.716066,2.153494,3.135203,9.998847,10.075669,0.716066,2.120434,3.243347,10.053597,10.075669,1,2,Cross Barr
2,2,05/04/19 4:30,1,0.687551,2.142630,3.066270,9.954641,10.081478,0.687551,2.105328,3.181119,10.001306,10.081478,1,2,Cross Barr
3,3,05/04/19 5:30,1,0.686916,2.140013,3.071850,9.956670,10.054274,0.686916,2.101800,3.191233,10.008420,10.054274,1,2,Cross Barr
4,4,05/04/19 6:30,1,0.698125,2.156244,3.063865,9.961413,10.091478,0.698125,2.117328,3.183534,10.009700,10.091478,1,2,Cross Barr


In [31]:
df1.columns

Index(['Unnamed: 0', 'time', 'cyl', 's_temp_crank', 'd_temp_crank',
       's_pressure_crank', 'd_pressure_crank', 'rpm_crank', 's_temp_mean',
       'd_temp_mean', 's_pressure_mean', 'd_pressure_mean', 'rpm_mean',
       'y_true', 'leak_level', 'station'],
      dtype='object')

In [0]:
df1 = df1.drop(columns='Unnamed: 0')
df1['y_true'] = df1['y_true'].fillna(0)
df1['leak_level'] = df1['leak_level'].fillna(0)

In [33]:
df1.head()

,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,y_true,leak_level,station
0,05/04/19 2:30,1,0.666534,2.080444,3.220836,10.105964,10.063987,0.666534,2.053527,3.307406,10.142075,10.063987,1,2,Cross Barr
1,05/04/19 3:30,1,0.716066,2.153494,3.135203,9.998847,10.075669,0.716066,2.120434,3.243347,10.053597,10.075669,1,2,Cross Barr
2,05/04/19 4:30,1,0.687551,2.142630,3.066270,9.954641,10.081478,0.687551,2.105328,3.181119,10.001306,10.081478,1,2,Cross Barr
3,05/04/19 5:30,1,0.686916,2.140013,3.071850,9.956670,10.054274,0.686916,2.101800,3.191233,10.008420,10.054274,1,2,Cross Barr
4,05/04/19 6:30,1,0.698125,2.156244,3.063865,9.961413,10.091478,0.698125,2.117328,3.183534,10.009700,10.091478,1,2,Cross Barr


In [34]:
df1.columns

Index(['time', 'cyl', 's_temp_crank', 'd_temp_crank', 's_pressure_crank',
       'd_pressure_crank', 'rpm_crank', 's_temp_mean', 'd_temp_mean',
       's_pressure_mean', 'd_pressure_mean', 'rpm_mean', 'y_true',
       'leak_level', 'station'],
      dtype='object')

In [0]:
X_columns = [     's_temp_mean',      'd_temp_mean',
        's_pressure_mean',  'd_pressure_mean',         'rpm_mean']

In [0]:
X = df1[X_columns]

# build  with mean

In [37]:
X

,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean
0,0.666534,2.053527,3.307406,10.142075,10.063987
1,0.716066,2.120434,3.243347,10.053597,10.075669
2,0.687551,2.105328,3.181119,10.001306,10.081478
3,0.686916,2.101800,3.191233,10.008420,10.054274
4,0.698125,2.117328,3.183534,10.009700,10.091478
...,...,...,...,...,...
165856,0.728417,2.854156,0.184073,2.375134,12.358284
165857,0.686059,2.810078,0.179784,2.370367,12.359520
165858,0.663322,2.785113,0.180072,2.373662,12.352143
165859,0.645405,2.767917,0.178354,2.373709,12.353393


In [0]:
y_true = df1.iloc[:,12]

In [39]:
y_true.unique()

array([1, 0])

In [0]:
seed = 42
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=test_size)

In [0]:
from collections import Counter

In [42]:
print(X_train.shape,y_train.shape)

(111126, 5) (111126,)


In [43]:
Counter(y_train)

Counter({0: 91856, 1: 19270})

64->16->4->1

In [0]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        # Number of input features is 12.
        self.layer_1 = nn.Linear(5, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_3 = nn.Linear(64, 10)
        self.layer_4 = nn.Linear(10, 4)
        self.layer_out = nn.Linear(4, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.batchnorm3 = nn.BatchNorm1d(10)
        #self.batchnorm4 = nn.BatchNorm1d(4)

        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [0]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [0]:
EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.0009

In [0]:
## train data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = trainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train.values))
## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = testData(torch.FloatTensor(X_test))

In [0]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [87]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [88]:
model = binaryClassification()
model.to(device)

binaryClassification(
  (layer_1): Linear(in_features=5, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=10, bias=True)
  (layer_4): Linear(in_features=10, out_features=4, bias=True)
  (layer_out): Linear(in_features=4, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [0]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [0]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [0]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        #X_batch = X_batch
  
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

In [0]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [93]:
confusion_matrix(y_test, y_pred_list)

array([[43643,  1578],
       [ 6413,  3101]])

In [94]:
y_test.size

54735

In [95]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92     45221
           1       0.66      0.33      0.44      9514

    accuracy                           0.85     54735
   macro avg       0.77      0.65      0.68     54735
weighted avg       0.84      0.85      0.83     54735



In [0]:
import pickle
filename = 'binarynn_model.pkl'

In [0]:
torch.save(model.state_dict(),filename)


In [98]:
model.load_state_dict(torch.load(filename))

<All keys matched successfully>

In [99]:
model.eval()

binaryClassification(
  (layer_1): Linear(in_features=5, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=10, bias=True)
  (layer_4): Linear(in_features=10, out_features=4, bias=True)
  (layer_out): Linear(in_features=4, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [0]:
y_pred_list = []
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())



In [0]:
predictions=[]
predictions.append([y_pred_list[i][0][0] for i in range(len(y_pred_list))])

In [102]:
print(classification_report(y_test, predictions[0]))

              precision    recall  f1-score   support

           0       0.87      0.97      0.92     45221
           1       0.66      0.33      0.44      9514

    accuracy                           0.85     54735
   macro avg       0.77      0.65      0.68     54735
weighted avg       0.84      0.85      0.83     54735

